# Controlled Gates 
<br/>

<center> 

Write a function controlled which takes a 2×2 matrix U representing a single qubit operator, and makes a 4×4 matrix which is a controlled variant of U, with the first argument being the control qubit.
Write a Quil program to define a controlled-Y gate in this manner. Find the wavefunction when applying this gate to qubit 1 controlled by qubit 0.

</center>


In [15]:
import math
import numpy as np
from pyquil.quil import Program
from pyquil.api import QVMConnection
from pyquil.gates import H

qvm = QVMConnection()

### What is a controlled gate? 
<br/>
Controlled gates act on 2 or more qubits, where one or more qubits act as a control for some operation.  So if the controlled qubit, qubit 0, is in state $\lvert 1 \rangle$, it preforms the operation to qubit 1.  Else do nothing.
<br/>
### How to create a controlled gate given gate U?
<br/>
Given U which looks as follows...
<br/>

$$
U =  \left(\begin{array}{cc} 
u_{00} & u_{01}\\
u_{10} & u_{11}
\end{array}\right)
$$
<br/>

The controlled version looks as follows...
<br/>

$$
C(U) =  \left(\begin{array}{cc} 
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0 \\
0 & 0 & u_{00} & u_{01}\\
0 & 0 & u_{10} & u_{11}
\end{array}\right)
$$
<br/>

This can be shown using kronecker products which can be found here https://en.wikipedia.org/wiki/Quantum_logic_gate#Controlled_(cX_cY_cZ)_gates
but for simplistic purposes will only show the end result.
Now we have all we need to create the function to make a controlled U gate!

In [18]:
def make_u_gate(u00, u01, u10, u11):
    
    #makes numPY array
    U = np.array(([u00, u01], 
                  [u10, u11]))
    return U;

def gates(gate_name):
    
    if(gate_name == "CX"):
        u_gate = make_u_gate(0.0, 1.0, 1.0, 0.0)
        
    elif(gate_name == "CY"):
        u_gate = make_u_gate(0.0, -1.0j, 1.0j, 0.0)
        
    elif(gate_name == "CZ"):
        u_gate = make_u_gate(1.0, 0.0, 0.0, -1.0)
        
    else:
        print("Must be a unitary matrix")
        u00 = float(input("u00: "))
        u01 = float(input("u01: "))
        u10 = float(input("u10: "))
        u11 = float(input("u11: "))
        u_gate = make_u_gate(u00, u01, u10, u11)
        
    return u_gate

def controlled(U):
    
    #create cU array
    cU = np.array(([1.0, 0.0, 0.0, 0.0], 
                   [0.0, 1.0, 0.0, 0.0],
                   [0.0, 0.0, U[0,0], U[0,1]],
                   [0.0, 0.0, U[1,0], U[1,1]]))
    
    #define the gate
    p = Program().defgate("c-u", cU)
    
    #put the bits initally in hadamard state
    p.inst(H(0))
    p.inst(H(1))
    
    #apply C(U)
    p.inst(("c-u", 0, 1))
    #calculate and print wavefucntion
    wavefunction = qvm.wavefunction(p)
    print("Wavefunction: ", wavefunction)


In [19]:
gate_name = input("Gate name(CX, CY, CZ, other): ")
U = gates(gate_name)
controlled(U)

Gate name(CX, CY, CZ, other): CY
Wavefunction:  (0.5+0j)|00> + -0.5j|01> + (0.5+0j)|10> + 0.5j|11>
